# Tips for Stan installation for Windows Users

*This recitation was written by Julian Wagner.*

<hr>

Here are two big issues I encountered when trying to install cmdstanpy on my Windows machines.

## A space in the username kills the install

Windows is terrible about keeping the user from introducing a space into the username, e.g. `Julian Wagner`. A space in the username leads to a space in the user folder, e.g. `/c/Users/Julian Wagner/`; I myself ended up with a space in the username for one of my Windows machines (curse you Windows for using a space!!), and it killed the install. I tried to make a workaround for this, but was not able to solve it. If you have a space in your user folder (you can check in `File explorer > Local disk (C:) > Users` and look at the folder there), I suggest you make a new user login for your machine that has no space. If you make this new user an admin, you will be able to access the files from your existing login on the new login. To make a new user login, you can do `Settings > Accounts > Other Users > Add someone else to this PC > I don't have this person's sign-in information > Add a user without a Microsoft account`. Now you can enter a new username and the key is to **pick a new username without a space!!**, e.g. `jwagner`. With that done, you can click on the new user and do `Change account type > Administrator` if you want to access files from your existing user with the new user. From the start menu, you can click on the little icon that looks like a person and click on the new username to switch to the new user login. It will take a bit to get set up for logging in. If you did a local install of Git Bash/Anaconda, then you will need to redo the setup from [bebi103a](https://bebi103a.github.io/lessons/00/index.html) and [bebi103b](https://bebi103b.github.io/lessons/00/setup.html) to do installs for the new user login. As irksome as it is, I am not aware of how to succeed with the install with a User folder with a space in it.

## Anaconda Navigator's terminal does not work for the install 

So far in the course, we have often used a terminal launched from within the Anaconda Navigator to do installs of packages. When attempting to install cmdstanpy on Windows (on two separate machines) from such a terminal I ran into this error message after running `python -c "import cmdstanpy; cmdstanpy.install_cmdstan()"`

```
Command "make build" failed
'cut' is not recognized as an internal or external command,
operable program or batch file.
'cut' is not recognized as an internal or external command,
operable program or batch file.
INFO: Could not find files for the given pattern(s).
The syntax of the command is incorrect.
mingw32-make: *** [bin/diagnose.exe] Error 1
```

which seems pretty cryptic and frustrating! As far as I can tell, this error means that the installation requires a Unix command called `cut` that is not available in the terminal launched through the Anaconda Navigator. Luckily, you should be able to get around this problem by instead running the install through Git Bash. I am assuming that you have installed [Git for Windows](https://gitforwindows.org/) as recommended by Justin in [bebi103a](https://bebi103a.github.io/lessons/00/setting_up_your_computer.html), which gets you access to the Git Bash app. When I initially installed Git Bash, commands like `conda` and `jupyter lab` and `python` did not work by default due to path issues. To fix this, from in the Git Bash terminal, I ran

```
/c/Users/jwagner/anaconda3/Scripts/conda init bash
```

If you have a global install of Anaconda instead of a local, the command will instead be something like
```
/c/ProgramData/Anaconda3/Scripts/conda init bash
```

In general, you will need to find the absolute path to the folder where Anaconda is installed, and the conda executable will be in the `/Scripts/` folder.

You can check that the `conda` command works by running `conda list` in the Git Bash terminal which will print out all the packages you have installed in your conda environment. Make sure you have the C++ toolchain installed (you can skip running this if you already ran it previously):

```
conda install libpython m2w64-toolchain -c msys2
```

You can now launch jupyter lab from within this Git Bash terminal by typing `jupyter lab`. From in this jupyter session, click `New Launcher` and under `Console` click `Python 3`

In this python console, run `import cmdstanpy` and then `cmdstanpy.install_cmdstan()`. This may take a few minutes.

NOTE: this command may look like it hangs (never finished running). Just let it run for a while, and then you can launch a new Jupyter notebook and try to run Justin's test script to see if the install worked.

The install took a couple of minutes to run for me, but this approach worked on both the Windows machines I tried it on.

## Justin's test code

To test your installation of CmdStanPy, you can run this test code on the famous "schools" hierarchical model.

In [1]:
import numpy as np

import cmdstanpy
import arviz as az

import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

schools_data = {
    "J": 8,
    "y": [28, 8, -3, 7, -1, 1, 18, 12],
    "sigma": [15, 10, 16, 11, 9, 11, 10, 18],
}

schools_code = """
data {
  int<lower=0> J; // number of schools
  vector[J] y; // estimated treatment effects
  vector<lower=0>[J] sigma; // s.e. of effect estimates
}

parameters {
  real mu;
  real<lower=0> tau;
  vector[J] eta;
}

transformed parameters {
  vector[J] theta = mu + tau * eta;
}

model {
  eta ~ normal(0, 1);
  y ~ normal(theta, sigma);
}
"""

with open("schools_code.stan", "w") as f:
    f.write(schools_code)

sm = cmdstanpy.CmdStanModel(stan_file="schools_code.stan")
samples = sm.sample(data=schools_data, output_dir="./")
samples = az.from_cmdstanpy(samples)

# Make a plot of samples
p = bokeh.plotting.figure(
    frame_height=250, frame_width=250, x_axis_label="μ", y_axis_label="τ"
)
p.circle(
    np.ravel(samples.posterior["mu"]),
    np.ravel(samples.posterior["tau"]),
    alpha=0.1
)

bokeh.io.show(p)

Loading BokehJS ...

INFO:cmdstanpy:compiling stan file /Users/bois/Dropbox/git/bebi103_course/2022/b/content/recitations/04/schools_code.stan to exe file /Users/bois/Dropbox/git/bebi103_course/2022/b/content/recitations/04/schools_code
INFO:cmdstanpy:compiled model executable: /Users/bois/Dropbox/git/bebi103_course/2022/b/content/recitations/04/schools_code
INFO:cmdstanpy:CmdStan start procesing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

INFO:cmdstanpy:CmdStan done processing.


In [2]:
%load_ext watermark
%watermark -v -p numpy,bokeh,cmdstanpy,arviz,jupyterlab

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.29.0

numpy     : 1.20.3
bokeh     : 2.3.3
cmdstanpy : 1.0.0
arviz     : 0.11.4
jupyterlab: 3.2.1

